## Make the GPT-2 simulate republican cases through Reinforcement Learning

In this homework, we will first train a reward model that assign higher reward to documents that sounds more like republican cases. Then we use RL to guide GPT-2 to complete the democratric cases in a republican way.

The reward model is covered in a previous notebook. All TODOs you need to finish lie in the RL part.

In [ ]:
%pip install transformers trl

In [2]:
# load sc_cases_cleaned.pkl that we used in the previous notebooks
# can be also find in https://github.com/elliottash/nlp_lss_2023/blob/master/notebooks/sc_cases_cleaned.pkl
from google.colab import files                                                                                                                                                                                       
uploaded = files.upload()

Saving sc_cases_cleaned.pkl to sc_cases_cleaned.pkl


In [3]:
import warnings; warnings.simplefilter('ignore')
import pandas as pd
import numpy as np

import torch
from tqdm import tqdm
import pandas as pd

tqdm.pandas()

from transformers import pipeline, AutoTokenizer, DistilBertTokenizerFast, DistilBertForSequenceClassification

from trl import PPOTrainer, PPOConfig, AutoModelForCausalLMWithValueHead
from trl.core import LengthSampler


### train a classification model as our reward model

In [4]:
# Read data, create numerical author-id
df = pd.read_pickle('sc_cases_cleaned.pkl', compression='gzip')
df = df.assign(author_id=(df['authorship']).astype('category').cat.codes)

# gpu or cpu?
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print (device)

# Load DistilBERT model for sequence classification using the Hugging Face library.
model_name = 'distilbert-base-uncased' # huggingface model_ID or path to folder 
model = DistilBertForSequenceClassification.from_pretrained(model_name)

# Tokenize 'opinion_text', converting it into model inputs as tensors (inputs) with padding and truncation.
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
inputs = tokenizer(df['opinion_text'].tolist(), return_tensors="pt", padding=True, truncation=True)

# Prepare labels by converting 'x_republican' into a PyTorch tensor (labels) of type 'long'.
labels = torch.tensor(df['x_republican'].tolist()).long()

# Set up the optimizer for training the model using the Adam optimizer from PyTorch (torch.optim.Adam). 
# The parameters of the DistilBERT model and the classifier are assigned different learning rates.
optimizer = torch.optim.Adam([
    {'params': model.distilbert.parameters(), 'lr': 1e-5},  
    {'params': model.classifier.parameters(), 'lr': 1e-3}
])

cuda


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['opinion_text'].tolist(), df['x_republican'].tolist(), test_size=.2)

# generate batches
X_train, X_test, y_train, y_test = np.array(X_train[:608]), np.array(X_test[:152]), np.array(y_train[:608]), np.array(y_test[:152])
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# Reshape training and testing sets into matrices with 8 columns and reshape corresponding labels accordingly
X_train, X_test, y_train, y_test = X_train.reshape(-1, 8), X_test.reshape(-1, 8), y_train.reshape(-1, 8), y_test.reshape(-1, 8)
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

X_train, X_test = X_train.tolist(), X_test.tolist()

(608,) (152,) (608,) (152,)
(76, 8) (19, 8) (76, 8) (19, 8)


In [6]:
X_train[0][0]

"Justice Scalia delivered the opinion of the Court.\n\nWe decide whether § 10(b) of the Securities Exchange Act of 1934 provides a cause of action to foreign plaintiffs suing  foreign and American defendants for misconduct in connection with securities traded on foreign exchanges.\n\nI \n\nRespondent National Australia Bank Limited (National) was, during the relevant time, the largest bank in Australia. Its Ordinary Shares--what in America would be called “common stock”--are traded on the Australian Stock Exchange Limited and on other foreign securities exchanges, but not on any exchange in the United States. There are listed on the New York Stock Exchange, however, National's American Depositary Receipts (ADRs), which represent the right to receive a specified number of National's Ordinary Shares. 547 F.3d 167, 168, and n. 1 (CA2 2008).\n\nThe complaint alleges the following facts, which we accept as true. In February 1998, National bought respondent HomeSide Lending, Inc., a mortgage

In [7]:
# Train the model for 10 epochs. Within the loop, it performs the following steps for each training example:
# Tokenizes the text input using the tokenizer defined earlier.
# Moves the model inputs and labels to the appropriate device (GPU or CPU).
# Performs a forward pass through the model and calculates the loss.
# Performs a backward pass and optimizer update.

from tqdm import tqdm

model.to(device)
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for text, labels in tqdm(zip(X_train, y_train), total=len(X_train)):
        # prepare model input through our tokenizer
        model_inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=256)
        # place everything on the right device
        model_inputs = {k:v.to(device) for k,v in model_inputs.items()}
        # labels have to be torch long tensors
        labels = torch.tensor(labels).long().to(device)
        # now, we can perform the forward pass
        output = model(**model_inputs, labels=labels)
        loss, logits = output[:2]
        # and the backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

torch.save(model, 'republican_classifier.pt')
republican_classifier = model

100%|██████████| 76/76 [00:15<00:00,  4.81it/s]


In [ ]:
pip install xformers

In [10]:
from transformers import DistilBertTokenizer, TextClassificationPipeline

# The pipeline takes the trained model republican_classifier, the tokenizer distil_tokenizer, and additional parameters 
# such as return_all_scores=True (to retrieve scores for all possible labels) and device=republican_classifier.device 
# (to ensure the pipeline runs on the same device as the trained model)
distil_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', max_length=512, truncation=True)
pipeline = TextClassificationPipeline(model=republican_classifier, tokenizer=distil_tokenizer, return_all_scores=True, device=republican_classifier.device)

# With this pipeline, you can now classify text inputs using the trained model. The pipeline will handle the tokenization, model inference, and label prediction for you.

In [11]:
# Get democrat texts (will be used later)
texts = df.loc[df['x_republican'] == 0, 'opinion_text'].tolist()
texts[0]

'JUSTICE GINSBURG delivered the opinion of the Court.\n\n A motion by a federal prisoner for postconviction relief under 28 U.S.C. § 2255 is subject to a one-year time limitation that generally runs from "the date on which the judgment of conviction becomes final." § 2255, P6(1). This case concerns the starting date for the one-year limitation. It presents a narrow but recurring question on which courts of appeals have divided: When a defendant in a federal prosecution takes an unsuccessful direct appeal from a judgment of conviction, but does not next petition for a writ of certiorari from this Court, does the judgment become "final" for postconviction relief purposes (1) when the appellate court issues its mandate affirming the conviction, or, instead, (2) on the date, ordinarily 69 days later, when the time for filing a petition for certiorari expires?\n\nIn accord with this Court\'s consistent understanding of finality in the context of collateral review, and the weight of lower co

In [13]:
X_test[0][0]

'Justice Scalia delivered the opinion of the Court. \n\nWe consider whether a District of Columbia prohibition on the possession of usable handguns in the home violates the Second Amendment to the Constitution. \n\n I \n\nThe District of Columbia generally prohibits the possession of handguns.  It is a crime to carry an unregistered  firearm, and the registration of handguns is prohibited.  See D. C. Code §§ 7-2501.01(12), 7-2502.01(a), 7-2502.02(a)(4) (2001).  Wholly apart from that prohibition, no person may carry a handgun without a license, but the chief of police may issue licenses for 1-year periods.  See §§ 22-4504(a), 22-4506.  District of Columbia law also requires residents to keep their lawfully owned firearms, such as registered long guns, "unloaded and dissembled or bound by a trigger lock or similar device" unless they are located in a place of business or are being used for lawful recreational activities.  See § 7-2507.02.1There are minor exceptions to all of these prohi

In [12]:
# Use the pipeline to perform text classification on a specific input.

# The `truncation=True` parameter indicates that if the text input exceeds the maximum sequence length specified during the tokenizer initialization,
# it will be truncated to fit within the limit.
# The pipeline will tokenize the text input using the `DistilBertTokenizer` object, perform classification using the trained model (`republican_classifier`), 
# and return the classification results.

pipeline(X_test[0][0], truncation=True)

[[{'label': 'LABEL_0', 'score': 9.167633834294975e-06},
  {'label': 'LABEL_1', 'score': 0.9999908208847046}]]

### Reinforcement Learning

In [14]:
# prepare the dataset for reinforcement learning

from torch.utils.data import Dataset

# Defines a custom dataset class named PPODataset that inherits from Dataset.
class PPODataset(Dataset):
  def __init__(self, tokenizer, texts, input_min_text_length, input_max_text_length):
    self.tokenizer = tokenizer
    self.texts = texts # democrat texts from above
    self.random_sample = LengthSampler(input_min_text_length, input_max_text_length)

  # Implement the __len__() method in the dataset class, which returns the length of the dataset (number of samples).
  def __len__(self):
    return len(self.texts)

  # Implement the __getitem__() method in the dataset class
  def __getitem__(self, index):
    # retrieve an individual sample from the dataset given its index
    text = self.texts[index]
    # create a dictionary
    sample = {} 
    # Assign the tokenized input IDs of the text as a PyTorch tensor to the key "input_ids". 
    # The tokenizer.encode() function is used to convert the text into input IDs, 
    # and the random_sample() function is called to determine the maximum length of the input. 
    sample["input_ids"] = torch.tensor(tokenizer.encode(text)[: self.random_sample()])
    # Assign the decoded text corresponding to the input IDs to the key "query". 
    sample["query"] = tokenizer.decode(sample["input_ids"])
    return sample

tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Create an instance of the PPODataset using the tokenizer, the texts, and specified minimum and maximum text lengths (10 and 15, respectively).
dataset = PPODataset(tokenizer, texts, 10, 15)

Now, the dataset object is ready to be used for reinforcement learning, where each sample in the dataset consists of an input text tokenized into input IDs and the corresponding decoded text.

In [18]:
dataset[0]

Token indices sequence length is longer than the specified maximum sequence length for this model (4480 > 1024). Running this sequence through the model will result in indexing errors


{'input_ids': tensor([25008,  8476,   402,  1268, 16811,  4261,    38,  6793,   262,  4459,
           286,   262,  3078,    13]),
 'query': 'JUSTICE GINSBURG delivered the opinion of the Court.'}

In [19]:
# prepare arguments and configurations for RL

# Define a dictionary that contains various keyword arguments for sentence scoring or evaluation during RL training.
sent_kwargs = {"return_all_scores": True, "function_to_apply": "none", "batch_size": 16}

# Set the minimum and maximum lengths for the generated output during RL training. 
output_min_length = 50
output_max_length = 100
output_length_sampler = LengthSampler(output_min_length, output_max_length)

# Define a dictionary that contains various keyword arguments for text generation during RL training. 
generation_kwargs = {
    "min_length": -1, # disable minimum length constraint
    "top_k": 0.0, # include all tokens during sampling
    "top_p": 1.0, # disable nucleus sampling 
    "do_sample": True, # enable sampling
    "pad_token_id": tokenizer.eos_token_id, 
}

# Define a collator function that takes a list of data samples and returns a dictionary containing the same keys as the data samples, 
# but with values as lists of the corresponding values from each sample. This function is used during data collation in RL training.
def collator(data):
  return dict((key, [d[key] for d in data]) for key in data[0])

# Specify RL configuration settings
config = PPOConfig(
    model_name="gpt2",
    learning_rate=1.41e-5,
    batch_size=32,
)

In [20]:
# TODO: prepare model (use gpt2), reference model, and PPO trainer for RL

model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)
ref_model = AutoModelForCausalLMWithValueHead.from_pretrained(config.model_name)

# PPO (Proximal Policy Optimization)
ppo_trainer = PPOTrainer(config, model, ref_model, tokenizer, dataset=dataset, data_collator=collator)

In [21]:
# TODO: conduct PPO training loop here. For efficiency, you can just train 3 batches.

# Iterates over the dataloader of ppo_trainer using tqdm to track the progress. 
# The loop is limited to a specified number of epochs (3 in this case) using the break statement.
for epoch, batch in tqdm(enumerate(ppo_trainer.dataloader), total=len(ppo_trainer.dataloader)):
    if epoch >= 3:
      break
    query_tensors = batch["input_ids"]

    #### Get response from gpt2: 
    # Generates responses for each query using the ppo_trainer.generate() method. 
    # The output_length_sampler() is used to determine the length of the generated responses within the specified range. 
    # The generated responses are stored in response_tensors.
    response_tensors = []
    for query in query_tensors:
        gen_len = output_length_sampler()
        generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **generation_kwargs)
        response_tensors.append(response.squeeze()[-gen_len:])
    # Adds the generated responses to the batch as "response" using the tokenizer.decode() method to convert the response tensors back into human-readable text.
    batch["response"] = [tokenizer.decode(r.squeeze()) for r in response_tensors]

    #### Compute score
    # Combines the input queries and generated responses to form texts for scoring. 
    # The queries and responses are concatenated in order, and the resulting texts are stored in texts.
    texts = [q + r for q, r in zip(batch["query"], batch["response"])]
    # Calculates scores for the texts using the pipeline() object and the specified sent_kwargs. The scores are stored in pipe_outputs as a list of dictionaries.
    pipe_outputs = pipeline(texts, **sent_kwargs)
    # Extracts the rewards (scores) from pipe_outputs 
    rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]

    #### Run PPO step
    # Performs a PPO step using ppo_trainer.step() by passing the query tensors, response tensors, and rewards as arguments. 
    # It returns training statistics, which are stored in stats.
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

 60%|██████    | 3/5 [02:12<01:28, 44.27s/it]


In [22]:
# visualize the outcomes generated by the RL tuned GPT-2

#### get a batch from the dataset
game_data = dict()
game_data["query"] = [dataset[i]["query"] for i in range(15)]
query_tensors = [dataset[i]["input_ids"] for i in range(15)]

response_tensors_ref, response_tensors = [], []
gen_kwargs = {"min_length": -1, "top_k": 0.0, "top_p": 1.0, "do_sample": True, "pad_token_id": tokenizer.eos_token_id}
#### get response from gpt2 and gpt2_ref
for i in range(15):
    gen_len = output_length_sampler()
    output = ref_model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors_ref.append(output)
    output = model.generate(
        torch.tensor(query_tensors[i]).unsqueeze(dim=0).to(device), max_new_tokens=gen_len, **gen_kwargs
    ).squeeze()[-gen_len:]
    response_tensors.append(output)

#### decode responses
game_data["response (before)"] = [tokenizer.decode(response_tensors_ref[i]) for i in range(15)]
game_data["response (after)"] = [tokenizer.decode(response_tensors[i]) for i in range(15)]

#### sentiment analysis of query/response pairs before/after
texts = [q + r for q, r in zip(game_data["query"], game_data["response (before)"])]
game_data["rewards (before)"] = [output[1]["score"] for output in pipeline(texts, **sent_kwargs)]

texts = [q + r for q, r in zip(game_data["query"], game_data["response (after)"])]
game_data["rewards (after)"] = [output[1]["score"] for output in pipeline(texts, **sent_kwargs)]

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results

,query,response (before),response (after),rewards (before),rewards (after)
0,JUSTICE GINSBURG delivered the opinion of the ...,"Court. Stevens, J., BRENNAN, POWELL, and BLAC...",Court:\n\nIn Misaraimo v. United States we he...,-4.216102,-3.843969
1,Justice Breyer delivered the opinion of the Co...,Court today held that the Fisher statement af...,Court of Appeals for the Second Circuit had a...,-4.295591,-3.850754
2,Justice Ginsburg delivered the opinion of the ...,"[Footnote 4/2]\n\nSee and\n\n`Cosby, 310 F.Sup...","Justice Breyer, joined by Justice Ginsburg, co...",-4.533293,-4.106676
3,Justice Ginsburg delivered the opinion of the ...,"Ginsburg offered the opinion on what he (""Kat...",Ginsburg joined the dissent in Sadler. Furthe...,-3.530821,-3.468459
4,Justice Breyer delivered the opinion of the Co...,\nBriar Stevens imposed on Kerr Urbandale Unit...,"\nCHIEF JUSTICE POWERS, joined by JUSTICE BLAC...",-4.590720,-4.194869
5,Justice Ginsburg delivered the opinion of the ...,One of the central constitutional challenges p...,Reobserver doctrine is explicitly established ...,-3.950624,-3.881912
6,Justice Breyer delivered the opinion of the Co...,ruling--and all subsequent decisions after it...,case arises from the trial court decisions co...,-4.194900,-3.432074
7,Justice Ginsburg delivered the opinion of the ...,case arose out of an appeal about a Dairy Que...,case deals largely with the politics of the s...,-3.905807,-3.631431
8,Justice Breyer delivered the opinion of the Co...,\n5. OVERSITY OF APPEAL UNDER LINCOLNQUER\n\nT...,\nJustice Breyer wrote that the NPRWorks Tribe...,-3.369586,-3.757503
9,Justice Ginsburg delivered the opinion of the ...,"\nWe continue with those comments, each confin...","\nI, join Mr. Justice REHNQUIST (ENDNOTES) and...",-4.116566,-4.385192


In [29]:
print(df_results['query'][0])
print('')
print('RESPONSE BEFORE')
print(df_results['response (before)'][0])
print('')
print('RESPONSE AFTER')
print(df_results['response (after)'][0])

JUSTICE GINSBURG delivered the opinion of the Court.

RESPONSE BEFORE
 Court. Stevens, J., BRENNAN, POWELL, and BLACKMUN delivered the opinion of the Court. BURGER, C. J., filed a dissenting opinion. BURGER, B., filed a dissenting opinion. POWELL, J., filed two dissenting opinions. STEVENS

RESPONSE AFTER
 Court:

In Misaraimo v. United States we held the trial court's grant of cause was reversed and the indictment-trial verdict was not reversed. in per id., the allegation that the jury did not "know" that the trial court had failed to place the believe r
